In [70]:
#importing libraries 
import os
import gc
import numpy as np 
import pandas as pd 
from math import * 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler 
from sklearn.tree import DecisionTreeRegressor 
import xgboost as xgb 
import matplotlib as plt 
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import mstats

In [5]:
class Config:
    LAG = 3
    VER = f'final_sub_v1'
    OUTPUT_DIR = './outputs'
    DATA_DIR = './data'
    DEBUG = True
    N_SPLITS = 5

In [259]:
train = pd.read_csv(os.path.join(Config.DATA_DIR,"Train.csv"), index_col=None)
print(train.shape)
train.head()

(7205, 9)


,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,"2,011",Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,"2,012",Silver,3.5,Nigerian Used,"125,000"
2,VHL19499,Lagos,Lexus,RX 350,"2,010",Red,9.2,Foreign Used,"110,852"
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,"2,017",Blue,22.8,Foreign Used,"30,000"
4,VHL12170,Ibadan,Toyota,Highlander,"2,002",Red,2.6,Nigerian Used,"125,206"


In [260]:
test = pd.read_csv(os.path.join(Config.DATA_DIR,"Test.csv"), index_col=None)
print(test.shape)
test.head()

(2061, 8)


,VehicleID,Location,Maker,Model,Year,Colour,Type,Distance
0,VHL18518,Abuja,BMW,323i,"2,008",White,Foreign Used,30524.0
1,VHL17149,Lagos,Toyota,Camry,"2,013",White,Foreign Used,NaN
2,VHL10927,Lagos,Toyota,Highlander Limited V6,"2,005",Gold,Foreign Used,NaN
3,VHL12909,Lagos,Toyota,Camry,"2,011",Gray,Foreign Used,166839.0
4,VHL12348,Lagos,Lexus,ES 350 FWD,"2,013",Red,Foreign Used,88862.0


In [261]:
submission = pd.read_csv(os.path.join(Config.DATA_DIR,"SampleSubmission.csv"), index_col=None)
submission.head()

,VehicleID,Amount (Million Naira)
0,VHL18518,1.0
1,VHL17149,1.0
2,VHL10927,1.0
3,VHL12909,1.0
4,VHL12348,1.0


In [262]:
submission['VehicleID'] = test['VehicleID']

In [266]:
train = train.rename(columns= {'Amount (Million Naira)':'Amount'})


In [267]:
train.drop_duplicates()

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,"2,011",Silver,2.20,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,"2,012",Silver,3.50,Nigerian Used,"125,000"
2,VHL19499,Lagos,Lexus,RX 350,"2,010",Red,9.20,Foreign Used,"110,852"
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,"2,017",Blue,22.80,Foreign Used,"30,000"
4,VHL12170,Ibadan,Toyota,Highlander,"2,002",Red,2.60,Nigerian Used,"125,206"
...,...,...,...,...,...,...,...,...,...
7200,VHL14329,Abuja,Honda,Civic,"2,018",Gray,5.70,Foreign Used,"65,000"
7201,VHL10637,Abuja,BMW,X3,"2,007",White,4.00,Nigerian Used,"200,000"
7202,VHL19734,Abuja,Toyota,RAV4 2.5 Limited 4x4,"2,010",Black,2.85,Nigerian Used,NaN
7203,VHL15569,Lagos,Mercedes-Benz,GLK-Class 350,"2,012",Black,8.65,Foreign Used,"85,750"


In [268]:
for i in train.columns:
    print(f"Unique {i}'s counts: {train[i].nunique()}")
    print(f"{train[i].unique()}\n")
    # print('{}'.format(overall[i].unique()))

Unique VehicleID's counts: 5141
['VHL12546' 'VHL18827' 'VHL19499' ... 'VHL12205' 'VHL17562' 'VHL15246']

Unique Location's counts: 3
['Abuja' 'Ibadan' 'Lagos']

Unique Maker's counts: 55
['Honda' 'Hyundai' 'Lexus' 'Mercedes-Benz' 'Toyota' 'Acura' 'Dodge'
 'Nissan' 'Kia' 'BMW' 'Volvo' 'Ford' 'Land Rover' 'Lincoln' 'Peugeot'
 'Chevrolet' 'Audi' 'Jaguar' 'Infiniti' 'Porsche' 'Fiat' 'Maserati'
 'Volkswagen' 'Suzuki' 'Bentley' 'GAC' 'Mazda' 'Scion' 'Renault'
 'Mitsubishi' 'Mini' 'Pontiac' 'Cadillac' 'Ferrari' 'Jeep' 'Buick'
 'Rolls-Royce' 'GMC' 'Chrysler' 'Lamborghini' 'Citroen' 'King' 'BAW'
 'Saturn' 'Tata' 'Opel' 'JAC' 'MG' 'Hummer' 'Subaru' 'Rover' 'Saab'
 'Skoda' 'IVM' 'Brabus']

Unique Model's counts: 1223
['Accord Coupe EX V-6' 'Sonata' 'RX 350' ... 'Almera 1.6 Lux'
 'X5 3.0i Sports Activity' '320i SV Premium']

Unique Year's counts: 33
['2,011' '2,012' '2,010' '2,017' '2,002' '2,019' '2,015' '2,004' '2,003'
 '2,008' '2,018' '2,020' '2,006' '2,016' '2,014' '2,007' '2,009' '2,005'
 '2,

In [271]:
train.shape,test.shape

((7205, 9), (2061, 8))

In [272]:
train.VehicleID.value_counts().head()

VHL18344    5
VHL14590    5
VHL12352    5
VHL16666    5
VHL19144    5
Name: VehicleID, dtype: int64

In [274]:
from sklearn.preprocessing import LabelEncoder 
columns_list = ["Maker", "Model", "Location", "Colour", "Type"] 
for var in columns_list:   
    lb = LabelEncoder()   
    full_var_data = pd.concat((train[var],test[var]),axis=0).astype('str')   
    temp = lb.fit_transform(np.array(full_var_data))   
    list(lb.classes_)   
    train[var] = lb.transform(np.array( train[var] ).astype('str'))   
    test[var] = lb.transform(np.array( test[var] ).astype('str'))

In [275]:
# Append test and train
train['is_train'] = 1
test['is_train'] = 0
overall = train.append(test, ignore_index = True)
overall

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train
0,VHL12546,0,17,142,"2,011",18,2.2,2,NaN,1
1,VHL18827,1,19,1204,"2,012",18,3.5,2,"125,000",1
2,VHL19499,2,29,1040,"2,010",17,9.2,1,"110,852",1
3,VHL17991,0,34,590,"2,017",3,22.8,1,"30,000",1
4,VHL12170,1,53,659,"2,002",17,2.6,2,"125,206",1
...,...,...,...,...,...,...,...,...,...,...
9261,VHL17903,0,53,186,"2,020",17,NaN,1,2650.0,0
9262,VHL14018,1,1,966,"2,011",3,NaN,1,99000.0,0
9263,VHL17473,1,53,317,"2,015",2,NaN,1,108000.0,0
9264,VHL11480,0,29,737,"2,013",22,NaN,1,52485.0,0


In [276]:
overall['Type'] = overall['Type'].ffill(axis=0)
overall['Type'] = overall['Type'].bfill(axis=0)

In [277]:
overall['Year'] = overall['Year'].ffill(axis=0)
overall['Year'] = overall['Year'].bfill(axis=0)

In [278]:
#overall['Amount'] = overall['Amount'].astype('str').map(lambda x: x.replace(',','').astype(np.int))
overall = overall.replace(',', '', regex = True)
#test = test.replace(',', '', regex = True)

In [279]:
overall['Year'] = overall['Year'].astype(float)

In [280]:
import statsmodels.api as sm 
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [281]:
overall['Amount'] = overall['Amount'].interpolate(method='linear', limit_direction='forward', axis=0)

In [282]:
overall['Distance'] = overall['Distance'].astype(float)
overall['Distance'] = overall['Distance'].fillna(overall['Distance'].dropna().mean())

In [283]:
combinations = [['Location','Maker'], ['Type','Model'], ['Year', 'Distance'], ['Model', 'Colour']] 
#combinations = [['Year'],['Model']] 
i = 0 
for combination in combinations: 
    target = 'Amount' 
    renamed_target = target + str(i) 
    new_cols = combination + [renamed_target] 
    temp_data = overall.groupby(combination).mean()[target].reset_index() 
    temp_data.columns = new_cols 
    overall = pd.merge(overall,temp_data,on = combination,how = 'left') 
    #test = pd.merge(test,temp_data,on = combination,how = 'left') 
    i+=1

In [284]:
overall.select_dtypes(include=np.number).loc[np.isinf(overall.select_dtypes(include=np.number)).any(1)]

,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3


In [285]:
overall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9266 entries, 0 to 9265
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   VehicleID  9266 non-null   object 
 1   Location   9266 non-null   int32  
 2   Maker      9266 non-null   int32  
 3   Model      9266 non-null   int32  
 4   Year       9266 non-null   float64
 5   Colour     9266 non-null   int32  
 6   Amount     9266 non-null   float64
 7   Type       9266 non-null   int32  
 8   Distance   9266 non-null   float64
 9   is_train   9266 non-null   int64  
 10  Amount0    9266 non-null   float64
 11  Amount1    9266 non-null   float64
 12  Amount2    9266 non-null   float64
 13  Amount3    9266 non-null   float64
dtypes: float64(7), int32(5), int64(1), object(1)
memory usage: 904.9+ KB


In [286]:
def missing_values_table(df):       
    mis_val = df.isnull().sum()       
    mis_val_percent = 100 * df.isnull().sum() / len(df)       
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)       
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})       
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)       
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"                
           "There are " + str(mis_val_table_ren_columns.shape[0]) + " columns that have missing values.")       
    return mis_val_table_ren_columns 

missing_values_table(overall)

Your selected dataframe has 14 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [287]:
overall

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3
0,VHL12546,0,17,142,2011.0,18,2.20,2,103332.682825,1,4.277409,2.200000,6.467374,2.200000
1,VHL18827,1,19,1204,2012.0,18,3.50,2,125000.000000,1,3.424667,3.235000,4.800000,4.872353
2,VHL19499,2,29,1040,2010.0,17,9.20,1,110852.000000,1,11.058355,7.465903,9.200000,9.004545
3,VHL17991,0,34,590,2017.0,3,22.80,1,30000.000000,1,17.192930,31.146914,23.908333,24.132500
4,VHL12170,1,53,659,2002.0,17,2.60,2,125206.000000,1,4.239292,4.064737,2.600000,9.860769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261,VHL17903,0,53,186,2020.0,17,3.38,1,2650.000000,0,8.390080,8.362909,3.380000,9.292000
9262,VHL14018,1,1,966,2011.0,3,3.38,1,99000.000000,0,3.380000,5.065000,3.640000,3.380000
9263,VHL17473,1,53,317,2015.0,2,3.38,1,108000.000000,0,4.239292,4.762460,3.380000,4.968495
9264,VHL11480,0,29,737,2013.0,22,3.38,1,52485.000000,0,11.577288,10.555000,3.380000,3.380000


In [291]:
overall

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3
0,VHL12546,0,17,142,2011.0,18,2.20,2,103332.682825,1,4.277409,2.200000,6.467374,2.200000
1,VHL18827,1,19,1204,2012.0,18,3.50,2,125000.000000,1,3.424667,3.235000,4.800000,4.872353
2,VHL19499,2,29,1040,2010.0,17,9.20,1,110852.000000,1,11.058355,7.465903,9.200000,9.004545
3,VHL17991,0,34,590,2017.0,3,22.80,1,30000.000000,1,17.192930,31.146914,23.908333,24.132500
4,VHL12170,1,53,659,2002.0,17,2.60,2,125206.000000,1,4.239292,4.064737,2.600000,9.860769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261,VHL17903,0,53,186,2020.0,17,3.38,1,2650.000000,0,8.390080,8.362909,3.380000,9.292000
9262,VHL14018,1,1,966,2011.0,3,3.38,1,99000.000000,0,3.380000,5.065000,3.640000,3.380000
9263,VHL17473,1,53,317,2015.0,2,3.38,1,108000.000000,0,4.239292,4.762460,3.380000,4.968495
9264,VHL11480,0,29,737,2013.0,22,3.38,1,52485.000000,0,11.577288,10.555000,3.380000,3.380000


In [292]:
overall.select_dtypes(include=np.number).loc[np.isinf(overall.select_dtypes(include=np.number)).any(1)]

,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3


In [298]:
overall["VehicleID_MeanPrice"] = overall.groupby(['VehicleID'])['Amount0'].transform('mean') 

In [299]:
overall["VehicleID_MinPrice"] = overall.groupby(['VehicleID'])['Amount0'].transform('min') 

overall["VehicleID_MaxPrice"] = overall.groupby(['VehicleID'])['Amount0'].transform('max') 

In [300]:
overall["Product_Maker_MaxPrice"] = overall.groupby(['Maker'])['Amount'].transform('max') 

overall["Product_Model_MeanPrice"] = overall.groupby(['Model'])['Amount'].transform('mean') 

In [302]:
overall['Distance'] = overall['Distance'].astype(float)
overall['Distance'] = overall['Distance'].fillna(overall['Distance'].dropna().mean())


In [303]:
overall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9266 entries, 0 to 9265
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   VehicleID                9266 non-null   object 
 1   Location                 9266 non-null   int32  
 2   Maker                    9266 non-null   int32  
 3   Model                    9266 non-null   int32  
 4   Year                     9266 non-null   float64
 5   Colour                   9266 non-null   int32  
 6   Amount                   9266 non-null   float64
 7   Type                     9266 non-null   int32  
 8   Distance                 9266 non-null   float64
 9   is_train                 9266 non-null   int64  
 10  Amount0                  9266 non-null   float64
 11  Amount1                  9266 non-null   float64
 12  Amount2                  9266 non-null   float64
 13  Amount3                  9266 non-null   float64
 14  VehicleID_MeanPrice     

In [307]:
overall.select_dtypes(include=np.number).loc[np.isinf(overall.select_dtypes(include=np.number)).any(1)]

,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3,VehicleID_MeanPrice,VehicleID_MinPrice,VehicleID_MaxPrice,Product_Maker_MaxPrice,Product_Model_MeanPrice


In [310]:
overall.select_dtypes(include=np.number).loc[np.isinf(overall.select_dtypes(include=np.number)).any(1)]

,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3,VehicleID_MeanPrice,VehicleID_MinPrice,VehicleID_MaxPrice,Product_Maker_MaxPrice,Product_Model_MeanPrice


In [317]:
overall.drop('VehicleID', inplace=True, axis=1)

In [318]:
overall

,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3,VehicleID_MeanPrice,VehicleID_MinPrice,VehicleID_MaxPrice,Product_Maker_MaxPrice,Product_Model_MeanPrice
0,0,17,142,2011.0,18,2.20,2,103332.682825,1,4.277409,2.200000,6.467374,2.200000,11.061922,4.277409,17.192930,19.0,2.200000
1,1,19,1204,2012.0,18,3.50,2,125000.000000,1,3.424667,3.235000,4.800000,4.872353,3.424667,3.424667,3.424667,36.0,4.392877
2,2,29,1040,2010.0,17,9.20,1,110852.000000,1,11.058355,7.465903,9.200000,9.004545,14.125643,11.058355,17.192930,195.0,7.189634
3,0,34,590,2017.0,3,22.80,1,30000.000000,1,17.192930,31.146914,23.908333,24.132500,17.192930,17.192930,17.192930,300.0,34.293895
4,1,53,659,2002.0,17,2.60,2,125206.000000,1,4.239292,4.064737,2.600000,9.860769,4.239292,4.239292,4.239292,187.0,9.144366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261,0,53,186,2020.0,17,3.38,1,2650.000000,0,8.390080,8.362909,3.380000,9.292000,8.390080,8.390080,8.390080,187.0,8.026232
9262,1,1,966,2011.0,3,3.38,1,99000.000000,0,3.380000,5.065000,3.640000,3.380000,8.498903,3.380000,11.058355,47.7,5.065000
9263,1,53,317,2015.0,2,3.38,1,108000.000000,0,4.239292,4.762460,3.380000,4.968495,4.239292,4.239292,4.239292,187.0,4.497698
9264,0,29,737,2013.0,22,3.38,1,52485.000000,0,11.577288,10.555000,3.380000,3.380000,11.577288,11.577288,11.577288,195.0,10.555000


In [321]:
num_var = ['Location', 'Maker', 'Model', 'Year', 'Colour',
       'Type', 'Distance', 'Amount0', 'Amount1', 'Amount2',
       'Amount3', 'VehicleID_MeanPrice', 'VehicleID_MinPrice',
       'VehicleID_MaxPrice', 'Product_Maker_MaxPrice',
       'Product_Model_MeanPrice']


In [322]:
def data_scaling( scaling_strategy , scaling_data , scaling_columns ):
    if scaling_strategy =="RobustScaler" :
        scaling_data[scaling_columns] = RobustScaler().fit_transform(scaling_data[scaling_columns])
    elif scaling_strategy =="StandardScaler" :
        scaling_data[scaling_columns] = StandardScaler().fit_transform(scaling_data[scaling_columns])
    elif scaling_strategy =="MinMaxScaler" :
        scaling_data[scaling_columns] = MinMaxScaler().fit_transform(scaling_data[scaling_columns])
    elif scaling_strategy =="MaxAbsScaler" :
        scaling_data[scaling_columns] = MaxAbsScaler().fit_transform(scaling_data[scaling_columns])
    else:  # If any other scaling send by mistake still perform Robust Scalar
        scaling_data[scaling_columns] = RobustScaler().fit_transform(scaling_data[scaling_columns])
    return scaling_data

# RobustScaler is better in handling Outliers :
scaling_strategy = ["RobustScaler", "StandardScaler", "MinMaxScaler", "MaxAbsScaler"]
overall = data_scaling(scaling_strategy[2], overall, num_var)

display(overall.head())
display(overall.columns)



,Location,Maker,Model,Year,Colour,Amount,Type,Distance,is_train,Amount0,Amount1,Amount2,Amount3,VehicleID_MeanPrice,VehicleID_MinPrice,VehicleID_MaxPrice,Product_Maker_MaxPrice,Product_Model_MeanPrice
0,0.0,0.309091,0.100638,0.744186,0.782609,2.2,0.666667,0.052046,1,0.010010,0.003633,0.014009,0.003842,0.030733,0.010010,0.049459,0.038821,0.003633
1,0.5,0.345455,0.853296,0.767442,0.782609,3.5,0.666667,0.062959,1,0.007406,0.005925,0.010127,0.009708,0.007406,0.007406,0.007406,0.076212,0.008490
2,1.0,0.527273,0.737066,0.720930,0.739130,9.2,0.333333,0.055833,1,0.030722,0.015297,0.020370,0.018778,0.040091,0.030722,0.049459,0.425932,0.014686
3,0.0,0.618182,0.418143,0.883721,0.130435,22.8,0.333333,0.015110,1,0.049459,0.067754,0.054611,0.051987,0.049459,0.049459,0.049459,0.656879,0.074725
4,0.5,0.963636,0.467045,0.534884,0.739130,2.6,0.666667,0.063063,1,0.009894,0.007763,0.005005,0.020658,0.009894,0.009894,0.009894,0.408336,0.019016


Index(['Location', 'Maker', 'Model', 'Year', 'Colour', 'Amount', 'Type',
       'Distance', 'is_train', 'Amount0', 'Amount1', 'Amount2', 'Amount3',
       'VehicleID_MeanPrice', 'VehicleID_MinPrice', 'VehicleID_MaxPrice',
       'Product_Maker_MaxPrice', 'Product_Model_MeanPrice'],
      dtype='object')

In [324]:
print(train.shape, test.shape)
df_train, df_test = overall[overall['is_train']==1], overall[overall['is_train']==0]
print(df_train.shape, df_test.shape)

(7205, 10) (2061, 9)
(7205, 18) (2061, 18)


In [325]:
y_train = df_train['Amount']

In [326]:
#X_train.info()
X_train = df_train[num_var]
#X_train.info()
X_test = df_test[num_var]

In [327]:

#RFE
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor


#RFE
lm = LinearRegression()
lm.fit(X_train,y_train)
#estimator = SVR(kernel="linear")
rfe = RFE(lm, n_features_to_select = 10)
rfe = rfe.fit(X_train, y_train)

In [328]:
#RFE
X_train.columns[rfe.support_]

Index(['Year', 'Distance', 'Amount0', 'Amount1', 'Amount2', 'Amount3',
       'VehicleID_MeanPrice', 'VehicleID_MinPrice', 'VehicleID_MaxPrice',
       'Product_Model_MeanPrice'],
      dtype='object')

In [329]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

[('Location', False, 3),
 ('Maker', False, 6),
 ('Model', False, 4),
 ('Year', True, 1),
 ('Colour', False, 5),
 ('Type', False, 7),
 ('Distance', True, 1),
 ('Amount0', True, 1),
 ('Amount1', True, 1),
 ('Amount2', True, 1),
 ('Amount3', True, 1),
 ('VehicleID_MeanPrice', True, 1),
 ('VehicleID_MinPrice', True, 1),
 ('VehicleID_MaxPrice', True, 1),
 ('Product_Maker_MaxPrice', False, 2),
 ('Product_Model_MeanPrice', True, 1)]

In [330]:
X_train_rfe = X_train[X_train.columns[rfe.support_]]
X_train_rfe.head()

,Year,Distance,Amount0,Amount1,Amount2,Amount3,VehicleID_MeanPrice,VehicleID_MinPrice,VehicleID_MaxPrice,Product_Model_MeanPrice
0,0.744186,0.052046,0.010010,0.003633,0.014009,0.003842,0.030733,0.010010,0.049459,0.003633
1,0.767442,0.062959,0.007406,0.005925,0.010127,0.009708,0.007406,0.007406,0.007406,0.008490
2,0.720930,0.055833,0.030722,0.015297,0.020370,0.018778,0.040091,0.030722,0.049459,0.014686
3,0.883721,0.015110,0.049459,0.067754,0.054611,0.051987,0.049459,0.049459,0.049459,0.074725
4,0.534884,0.063063,0.009894,0.007763,0.005005,0.020658,0.009894,0.009894,0.009894,0.019016


In [331]:
#RFE
def build_model(X,y):
    X = sm.add_constant(X) #Adding the constant
    lm = sm.OLS(y,X).fit() # fitting the model
    print(lm.summary()) # model summary
    return X

In [332]:

def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

#RFE
X_train_new = build_model(X_train_rfe,y_train)   

                            OLS Regression Results                            
Dep. Variable:                 Amount   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                 1.103e+04
Date:                Wed, 29 Jun 2022   Prob (F-statistic):               0.00
Time:                        16:58:33   Log-Likelihood:                -23436.
No. Observations:                7205   AIC:                         4.689e+04
Df Residuals:                    7194   BIC:                         4.697e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [333]:
#RFE
X_train_new = X_train_rfe.drop(["Distance"], axis = 1)

#RFE
X_train_new = build_model(X_train_new, y_train)  

                            OLS Regression Results                            
Dep. Variable:                 Amount   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                 1.226e+04
Date:                Wed, 29 Jun 2022   Prob (F-statistic):               0.00
Time:                        16:58:35   Log-Likelihood:                -23436.
No. Observations:                7205   AIC:                         4.689e+04
Df Residuals:                    7195   BIC:                         4.696e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [334]:
#Calculating the Variance Inflation Factor
checkVIF(X_train_new)

,Features,VIF
6,VehicleID_MeanPrice,62.18
0,const,60.33
7,VehicleID_MinPrice,21.44
9,Product_Model_MeanPrice,21.33
8,VehicleID_MaxPrice,17.21
5,Amount3,16.24
3,Amount1,14.80
2,Amount0,5.69
4,Amount2,2.92
1,Year,1.45


In [335]:
#RFE
X_train_new = X_train_new.drop(["VehicleID_MeanPrice"], axis = 1)

#RFE
X_train_new = build_model(X_train_new, y_train)

                            OLS Regression Results                            
Dep. Variable:                 Amount   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                 1.369e+04
Date:                Wed, 29 Jun 2022   Prob (F-statistic):               0.00
Time:                        16:58:40   Log-Likelihood:                -23461.
No. Observations:                7205   AIC:                         4.694e+04
Df Residuals:                    7196   BIC:                         4.700e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [336]:
#Calculating the Variance Inflation Factor
checkVIF(X_train_new)

,Features,VIF
0,const,60.21
8,Product_Model_MeanPrice,21.29
5,Amount3,16.12
3,Amount1,14.79
2,Amount0,5.16
4,Amount2,2.89
7,VehicleID_MaxPrice,2.38
6,VehicleID_MinPrice,2.36
1,Year,1.45


In [337]:
lm = sm.OLS(y_train,X_train_new).fit()
# Now let's use our model to make predictions.
X_train_new = X_train_new.drop('const',axis=1)
# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train_new.columns]

# Adding a constant variable 
X_test_new = sm.add_constant(X_test_new)

In [338]:
X_test_new

,const,Year,Amount0,Amount1,Amount2,Amount3,VehicleID_MinPrice,VehicleID_MaxPrice,Product_Model_MeanPrice
7205,1.0,0.674419,0.048103,0.006247,0.006821,0.006432,0.030722,0.048103,0.006247
7206,1.0,0.790698,0.018927,0.009309,0.017497,0.010748,0.018927,0.049459,0.008723
7207,1.0,0.604651,0.018927,0.008136,0.006468,0.005806,0.018927,0.018927,0.007170
7208,1.0,0.744186,0.018927,0.009309,0.006821,0.009307,0.010010,0.022572,0.008723
7209,1.0,0.790698,0.030722,0.020495,0.006821,0.018003,0.030722,0.030722,0.020221
...,...,...,...,...,...,...,...,...,...
9261,1.0,0.953488,0.022572,0.017284,0.006821,0.019410,0.022572,0.022572,0.016539
9262,1.0,0.744186,0.007269,0.009979,0.007426,0.006432,0.007269,0.030722,0.009979
9263,1.0,0.837209,0.009894,0.009309,0.006821,0.009919,0.009894,0.009894,0.008723
9264,1.0,0.790698,0.032307,0.022140,0.006821,0.006432,0.032307,0.032307,0.022140


In [339]:

test_pred = lm.predict(X_test_new)

In [344]:
ty = round(test_pred, 2)

In [353]:
# Round off the Probability Results :
predictions = [round(value, 2) for value in test_pred]
# Create a Dataframe Table for Submission Purpose :
Result_Promoted = pd.DataFrame({'VehicleID': submission['VehicleID'], 'Amount (Million Naira)' : predictions})
Result_Promoted.to_csv("zik_stat_v1_4.csv", index=False)


In [348]:
submission['Amount (Million Naira)'] = ty
submission.to_csv('zik_stat_v1_4.csv', index =  False)

In [363]:
X_train_new

,Year,Amount0,Amount1,Amount2,Amount3,VehicleID_MinPrice,VehicleID_MaxPrice,Product_Model_MeanPrice
0,0.744186,0.010010,0.003633,0.014009,0.003842,0.010010,0.049459,0.003633
1,0.767442,0.007406,0.005925,0.010127,0.009708,0.007406,0.007406,0.008490
2,0.720930,0.030722,0.015297,0.020370,0.018778,0.030722,0.049459,0.014686
3,0.883721,0.049459,0.067754,0.054611,0.051987,0.049459,0.049459,0.074725
4,0.534884,0.009894,0.007763,0.005005,0.020658,0.009894,0.009894,0.019016
...,...,...,...,...,...,...,...,...
7200,0.906977,0.010010,0.007578,0.025725,0.007181,0.008801,0.010010,0.006911
7201,0.651163,0.048103,0.006819,0.008264,0.008378,0.007063,0.048103,0.010873
7202,0.720930,0.022572,0.005073,0.011904,0.005850,0.022572,0.049459,0.008023
7203,0.767442,0.032729,0.015063,0.019090,0.013309,0.018927,0.032729,0.014701


In [ ]:
num_var = ['Amount1', 'Year', 'Type', 'Product_Model_MeanPrice', 'Location', 'Maker']

Xt, X_test = df_train[num_var], df_test[num_var]

scaler = MinMaxScaler()

Xt = scaler.fit_transform(Xt)
X_test = scaler.fit_transform(X_test)

In [504]:
r1 = XGBRegressor(random_state = 42)
r2 = RandomForestRegressor(random_state=42)
r3 = CatBoostRegressor(verbose=False,random_state = 42)
r4 = LGBMRegressor(random_state=42)

model = VotingRegressor([('r1', r1),('r2', r2),('r3',r3), ('r4', r4)],weights=(1,1,1,1))
model.fit(X_train_new,y_train)
#val_pred = model.predict(X_test)
#np.mean(mean_squared_error(y_val,val_pred))



In [492]:
# predict the results
y_pred=clf.predict(X_test)

In [493]:
submission['Amount (Million Naira)'] = y_pred
submission.to_csv('lgb_modelv1.csv', index=False) 

In [ ]:
clfxgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)

le = LabelEncoder()
y = le.fit_transform(y)

y_pred = xgb.predict(X_test)

clfxgb.fit(Xt, y,
            #eval_set=[(X_val, y_val)],
            early_stopping_rounds=50,
            eval_metric = 'rmse',
            verbose=False
         )
y_pred_trn = clfxgboost.predict(X_test)

In [415]:


clfxgb = XGBClassifier()
clfxgb.fit(X_2,y)
feature_important = clfxgb.get_booster().get_score(importance_type='gain') 
keys = list(feature_important.keys()) 
values = list(feature_important.values()) 
total = sum(values) 
new = [value * 100. / total for value in values] 
new = np.round(new,2) 
feature_importances = pd.DataFrame() 
feature_importances['Features'] = keys 
feature_importances['Importance (%)'] = new 
feature_importances = feature_importances.sort_values(['Importance (%)'],ascending=False).reset_index(drop=True) 
feature_importances 

feature_importances.style.set_properties(**{'font-size':'10pt'})

,Features,Importance (%)
0,Amount1,18.500000
1,Year,9.480000
2,Type,7.570000
3,Product_Model_MeanPrice,7.510000
4,Location,6.750000
5,Maker,6.170000
6,Product_Maker_MaxPrice,6.040000
7,Amount0,6.020000
8,Distance,5.240000
9,Model,4.970000


In [413]:
feature_important

{'f0': 0.5207287669181824,
 'f1': 0.3103378415107727,
 'f2': 0.23272700607776642,
 'f3': 0.23380133509635925,
 'f4': 0.39389318227767944}